# Cálculo da matriz de capacitância em derivação

In [1]:
import numpy as np
%precision 4

u'%.4f'

Determine the shunt admittance matrix for the overhead line in Example 4.1.
Assume that the neutral conductor is 25 ft. above ground.

Dados dos condutores:

**336,400 26/7 ACSR:**

**d_c = 0.721 inches, RD_c = 0.03004 ft.,**

**4/0 6/1 ACSR:**

**d_s = 0.563 inches, RD_s = 0.02346 ft.**

<img src="Figuras/fig_021.png"/>

In [2]:
rdc = 0.03004
rds = 0.02346

## Montagem da matriz de distâncias entre os condutores

In [4]:
dab = 2.5
dbc = 4.5
dca = 7.0
dan = 5.6569
dbn = 4.272
dcn = 5.0

In [5]:
D = np.array([[0.0, dab, dca, dan],
              [dab, 0.0, dbc, dbn],
              [dca, dbc, 0.0, dcn],
              [dan, dbn, dcn, 0.0]])
print D

[[ 0.      2.5     7.      5.6569]
 [ 2.5     0.      4.5     4.272 ]
 [ 7.      4.5     0.      5.    ]
 [ 5.6569  4.272   5.      0.    ]]


## Montagem da matriz de distâncias entre os condutores e suas imagens

In [6]:
S = np.array([[58.0000, 58.0539, 58.4209, 54.1479],
              [58.0539, 58.0000, 58.1743, 54.0208],
              [58.4209, 58.1743, 58.0000, 54.0833],
              [54.1479, 54.0208, 54.0833, 50.0000]])
print S

[[ 58.      58.0539  58.4209  54.1479]
 [ 58.0539  58.      58.1743  54.0208]
 [ 58.4209  58.1743  58.      54.0833]
 [ 54.1479  54.0208  54.0833  50.    ]]


## Cálculo da matriz de diferença de potencial primitiva

Considerando os coeficientes de diferença de potencial própria e múltua:

\begin{equation}
    V_{ig} = \hat P_{ii} \cdot q_i + \hat P_{ij} \cdot q_j
\end{equation}

Se considerarmos para as linhas aéreas:

\begin{eqnarray}
        \epsilon_{ar} & = & 1,0 \times 8,85 \times 10^{-12} ~~ [F/metro] \\
        \epsilon_{ar} & = & 1,424 \times 10^{-2} \mu ~~ [F/milha]
\end{eqnarray}

Assim ficamos com:

\begin{eqnarray}
        P_{ii} & = & 11,17689 \cdot \ln \frac{S_{ii}}{RD_{i}} ~~ [milha/\mu F] \\
        P_{ij} & = & 11,17689 \cdot \ln \frac{S_{ij}}{D_{ij}} ~~ [milha/\mu F]
\end{eqnarray}

In [7]:
pabc = list()
for i in range(4):
    pl = list()
    for j in range(4):
        if i == j:
            if i == 3:
                pf = 11.17689 * np.log(S[i,j]/rds)
            else:
                pf = 11.17689 * np.log(S[i,j]/rdc)
        else:
            pf = 11.17689 * np.log(S[i,j]/D[i,j])
        pl.append(pf)
    pabc.append(pl)
P = np.array(pabc)

print P

[[ 84.5606  35.1522  23.7147  25.2468]
 [ 35.1522  84.5606  28.6058  28.359 ]
 [ 23.7147  28.6058  84.5606  26.6132]
 [ 25.2468  28.359   26.6132  85.6651]]


## Particionamento da matriz de impedâncias primitiva para redução de Kron

In [8]:
pij = P[0:3, 0:3]
pij = np.reshape(pij, (3, 3))
print pij

[[ 84.5606  35.1522  23.7147]
 [ 35.1522  84.5606  28.6058]
 [ 23.7147  28.6058  84.5606]]


In [9]:
pin = P[0:3, 3]
pin = np.reshape(pin, (3, 1))
print pin

[[ 25.2468]
 [ 28.359 ]
 [ 26.6132]]


In [10]:
pnn = P[3,3]
pnn = np.reshape(pnn, (1, 1))
print pnn

[[ 85.6651]]


In [11]:
pnj = P[3, 0:3]
pnj = np.reshape(pnj, (1, 3))
print pnj

[[ 25.2468  28.359   26.6132]]


## Aplicação da redução de Kron

A equação que representa a redução de Kron é dada por:

\begin{equation}
        \mathbf{\hat P_{abc}} = (\mathbf{\hat p_{ij}} - \mathbf{\hat p_{in}} \cdot \mathbf{\hat p^{-1}_{nn}} \cdot \mathbf{\hat p_{nj}} )
\end{equation}

In [12]:
Pabc = pij - np.dot(np.dot(pin, np.linalg.inv(pnn)) ,pnj)
Pabc

array([[ 77.12  ,  26.7944,  15.8714],
       [ 26.7944,  75.1726,  19.7956],
       [ 15.8714,  19.7956,  76.2929]])

## Cálculo da matriz de capacitância

O inverso da matriz de potencial de fase, será a matriz de capacitância de fase:

\begin{equation}
    \mathbf{C_{abc}} = \mathbf{P_{abc}}^{-1}
\end{equation}

In [13]:
Cabc = np.linalg.inv(Pabc)
Cabc

array([[ 0.015 , -0.0049, -0.0019],
       [-0.0049,  0.0159, -0.0031],
       [-0.0019, -0.0031,  0.0143]])

## Cálculo da matriz de admitância em derivação

A matriz de admitância em derivação pode ser obtida:

\begin{equation}
    \mathbf{y_{abc}} = \mathbf{0} + j \cdot \omega \cdot \mathbf{C_{abc}} ~~ [\mu S/milha]
\end{equation}

Em que:

\begin{equation}
    \omega = 2 \cdot \pi \cdot f = 2 \cdot \pi \cdot 60 = 376,9911
\end{equation}

In [14]:
yabc = 1.0j * (2*np.pi*60.0) * Cabc
yabc

array([[ 0.+5.6711j, -0.-1.8362j, -0.-0.7033j],
       [-0.-1.8362j,  0.+5.9773j, -0.-1.1689j],
       [-0.-0.7033j, -0.-1.1689j,  0.+5.391j ]])